In [466]:
import pandas as pd
import numpy as np
import json
from dotenv import load_dotenv
import os 
import base64
import json
from requests import post, get

import time
import numpy as np
import pandas as pd


#loads env vars located in .env file
load_dotenv()

#get and print client id and secret
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
print(client_id, client_secret)

ROOT_API_URL = "https://api.spotify.com/v1"



def str_list_to_array(str_list):
    return np.array([json.loads(v) for v in str_list])

track_info = pd.read_csv('./data/AudioFeats_MustBeThePlace.csv')
audio_df1 = pd.read_csv('./data/aud_analys_data_MustBePlace.csv')
audio_df1.head(3)

5835805e22a345948373bd954162324c 6ccb11160d65423c96d93038b719b902


,track_name,track_id,start,duration,end,confidence,loudness_start,loudness_max_time,loudness_max,loudness_end,attack_aggression,pitches,timbre
0,This Must Be the Place (Naive Melody) - 2005 R...,6aBUnkXuCEQQHAlTokv9or,0.00000,0.13392,0.13392,0.0,-60.000,0.00000,-60.000,0.0,NaN,"[1.0, 0.977, 0.713, 0.71, 0.654, 0.626, 0.561,...","[0.011, 170.887, 9.19, -28.789, 57.303, -50.12..."
1,This Must Be the Place (Naive Melody) - 2005 R...,6aBUnkXuCEQQHAlTokv9or,0.13392,0.28408,0.41800,1.0,-60.000,0.03978,-10.570,0.0,1242.584213,"[0.266, 0.214, 0.459, 0.851, 1.0, 0.442, 0.382...","[33.232, 73.877, -196.871, 126.219, 17.257, 11..."
2,This Must Be the Place (Naive Melody) - 2005 R...,6aBUnkXuCEQQHAlTokv9or,0.41800,0.27814,0.69614,1.0,-42.018,0.01052,-6.792,0.0,3348.479087,"[0.26, 0.194, 0.253, 0.428, 0.67, 0.449, 0.498...","[39.723, 89.491, -175.686, 179.658, -26.575, -..."


In [467]:
def get_token():
    auth_string = client_id+":"+client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url="https://accounts.spotify.com/api/token"
    headers= {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    js_result = json.loads(result.content)
    token=js_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}


In [468]:
# timbre_vals=str_list_to_array(audio_df1.timbre.tolist())
# timbre_df = pd.DataFrame(data=timbre_vals, columns=['timbre_%d'%i for i in range(timbre_vals.shape[1])])
scale_timbre=True
timbre_vals=[]
pitch_vals=[]
for track_id in track_info.track_id.tolist():
    timbre = audio_df1[audio_df1.track_id==track_id].timbre.values.tolist()
    timbre = str_list_to_array(timbre)
    if scale_timbre:
        timbre = (timbre - timbre.min(0)) / (timbre.max(0) - timbre.min(0))
    timbre_vals += [timbre]

In [469]:
CC = []
C = np.ones((len(timbre_vals),len(timbre_vals))) * np.inf

for k in range(12):
    for i in range(len(timbre_vals)):
        for j in range(i+1,len(timbre_vals)):
            if len(timbre_vals[i]) > len(timbre_vals[j]):
                x = timbre_vals[i]
                y = timbre_vals[j]
            else:
                x = timbre_vals[j]
                y = timbre_vals[i]
                
            
            # Calculate the maximum possible lag
            max_lag = len(x) - len(y) + 1
            
            # Initialize array to store cross-correlation values for each lag
            cross_corr_values = []
            
            # Calculate cross-correlation for each possible alignment
            for lag in range(max_lag):
                # Select the corresponding segment of y
                x_segment = x[:,k][lag:lag + len(y)]
                # Calculate cross-correlation between x and the selected segment of y
                cross_corr_value = np.correlate(y[:,k], x_segment)[0]
                cross_corr_values.append(cross_corr_value)
            cross_corr_values=np.array(cross_corr_values)
            # print(i, j)
            C[i,j] = cross_corr_values.mean()
    CC += [C]
        

KeyboardInterrupt: 

In [ ]:
CC_aggregate = np.array(CC).mean(0)
CC_aggregate

In [ ]:
np.argmax(np.nan_to_num(CC_aggregate, posinf=0), axis=1)
ix_inspect = [np.unravel_index(ix, CC_aggregate.shape) for ix in np.argsort(np.nan_to_num(CC_aggregate, posinf=0).flatten())[::-1][:4]]
track_names_dis = track_info.iloc[list(sum(ix_inspect, ())),:].track_name.unique().tolist()
track_ids_dis = track_info.iloc[list(sum(ix_inspect, ())),:].track_id.unique().tolist()
track_names_dis

In [ ]:
unique, counts = np.unique(np.argmin(CC_aggregate, axis=1), return_counts=True)
print(unique)
print(counts)

In [ ]:
track_info.iloc[[ 0, 23, 35, 40, 41, 45, 63, 71, 74, 76],:]

In [ ]:
# np.unravel_index(np.argmin(CC_aggregate), CC_aggregate.shape)
ix_inspect = [np.unravel_index(ix, CC_aggregate.shape) for ix in np.argsort(CC_aggregate.flatten())[:2]]
print(ix_inspect)
track_info.iloc[list(sum(ix_inspect, ())),:]

In [ ]:
np.argsort(np.nan_to_num(CC_aggregate[[35, 40, 23]], posinf=0).flatten())[::-1][:2]

In [ ]:

np.argmax(np.nan_to_num(CC_aggregate[[35, 40, 23]], posinf=0), axis=1)
track_names = track_info.iloc[list(sum(ix_inspect, ()))+[50],:].track_name.unique().tolist()
track_ids = track_info.iloc[list(sum(ix_inspect, ()))+[50],:].track_id.unique().tolist()
track_info.iloc[list(sum(ix_inspect, ()))+[50],:]

In [ ]:
track_names=track_info.iloc[list(sum(ix_inspect, ())),:].track_name.unique().tolist()
track_ids = track_info.iloc[list(sum(ix_inspect, ())),:].track_id.unique().tolist()
print(track_names)

In [ ]:
track_names_dis
json.loads(result.content)

In [483]:
token=get_token()
print(token)
headers = get_auth_header(token)

BQC4SzpLTvClKmpvtSdsRpsbFibG-exS4TdFWmojc1qtTwmJTheTYeY-xXIJrmOrtr1Gj3O1pOUf8agelzxTuSBqaM-O_lgyz-V1msc9zshROGHBID4


In [ ]:
track_dict = {'artist_name': ['Sasha', 'Sahar', 'Blackcloud'],
              'artist_id': ['6u4jLGLPuarS3i2XWHVxoS', '3b7TXhgeS7YpSumMEwbFJO', '6K0Y096jSf23jHzPtClzPT'],
              'track_name': ['Dry & High', 'Egomania', 'Six Nine - Wolfson Remix'],
              'track_id': ['284FKz13AitOQjOVivE7ra', '1M7c6Ye67O9x5x9UoDXVky', '0pThX0xfbMPk0W8kGqZgc4']
}

artist_ids = track_dict['artist_id']
track_ids = track_dict['track_id']+['5BG5t49EldKlQsRAMGlUv0', '66ftD6ZsTzLLxYvN5l1vyr']

track_names = ['Shine', 'Egomania']
track_ids = ['3rQGZruQVgRv9ulpLYaKED', '1M7c6Ye67O9x5x9UoDXVky', '0pThX0xfbMPk0W8kGqZgc4']
pd.DataFrame(track_dict)

In [492]:


# url = f"https://api.spotify.com/v1/recommendations?limit=10&seed_tracks={'%2C'.join(track_ids)}&max_popularity=20"
url = f"https://api.spotify.com/v1/recommendations?limit=5&seed_artists=6u4jLGLPuarS3i2XWHVxoS&seed_tracks={'%2C'.join(track_ids)}&max_popularity=100"

result = get(url, headers=headers)
js_result = json.loads(result.content)['tracks']


In [493]:
artists = [res['artists'][0]['name']  for res in js_result]
df_track_recs = pd.DataFrame(js_result)[['name', 'popularity']]
df_track_recs.insert(0, 'artist', artists)
df_track_recs['query'] = [f"artist:\042{a}\042 track:\042{t}\042" for a,t in zip(df_track_recs['artist'].tolist(), df_track_recs['name'].tolist())]
pd.set_option('display.max_columns', None)  
with pd.option_context('display.max_colwidth', None):
    # display(df_track_recs.sort_values('popularity', ascending=True).head(50)[['popularity', 'query']])
    display(df_track_recs[['popularity', 'query']])

,popularity,query
0,19,"artist:""Richie Blacker"" track:""Sesh Gremlins"""
1,21,"artist:""Alphalife"" track:""Danke Funk"""
2,20,"artist:""Sasha"" track:""Mongoose - Guy J Remix"""
3,20,"artist:""Tal Fussman"" track:""The Unknown"""
4,16,"artist:""Prana Flow"" track:""Stereovision"""
